### 이진 분류기 훈련하기
간단한 분류 모델을 훈련시키기 -> 타깃 벡터의 값이 두 개 뿐인 이진 분류기 사용. 
- 로지스틱 함수는 함수의 출력을 0과 1사이로 제한하는 효과가 있다. 
- 함수: $P(y_i = 1 | X)=\frac{1}{1+e^-(\beta_0+\beta_1*x)}$
- $P(y_i = 1 | X)$가 0.5보다 크면 1로 예측하고 그렇지 않으면 0으로 예측한다.

사이킷런에서 로지스틱 회귀 모델을 훈련시킨다면 새로운 샘플의 클래스를 예측할 수 있다.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data[:100,:]
target = iris.target[:100]

#feature 표준화
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 로지스틱 회귀 모델 생성
logistic_regression = LogisticRegression(random_state=20200920)
model = logistic_regression.fit(features_standardized, target)

#새로운 샘플
new_observation = [[.5, .5, .5, .5]]
print(model.predict(new_observation), model.predict_proba(new_observation))

[1] [[0.17738424 0.82261576]]


new_observation이라는 샘플을 클래스 1로 예측하였고 그러한 근거를 predict_proba를 통해 확인할 수 있다. 0일 확률은 17.7%이고 1일 확률은 82.3%이다.

### 다중 클래스 분류기 훈련하기
두 개보다 많은 클래스가 있을 때 분류 모델을 훈련해야 한다. ->  OVR(one-vs-rest) 또는 다중 분류 기법으로 사이킷런의 LogisticRegression 사용한다.

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# 데이터
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 데이터 표준화
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# OVR 로지스틱 회귀 모델
logistic_regression = LogisticRegression(random_state=0, multi_class="ovr")
model = logistic_regression.fit(features_standardized, target)   # fit method에는 feature와 target을 순서대로 입력하면 된다.

#새로운 샘플
new_observation = [[.5, .5, .5, .5]]
print(model.predict(new_observation), model.predict_proba(new_observation))

[2] [[0.0387617  0.40669108 0.55454723]]


클래스가 두 개보다 많은 타깃 벡터를 다루는 방법
- OVR 로지스틱 회귀: 각 클래스마다 모델을 만들어 샘플이 해당 클래스에 속하는지 여부를 예측, 모델마다 예측(분류)할 때 독립적이라고 가정.
- 다항 로지스틱 회귀(MLR): 다항로지스틱회귀의 경우 결과 항이 3개 이상인 경우로 로지스틱 회귀분석과 유사하지만 종속변수가 두 개의 범주로 제한되지 않고 일반적으로 사용할 수 있다.

LogisticRegression는 최적화 알고리즘을 지정하는 solver 매개변수를 사용한다. solver 매개변수는 기본값이 lbfgs이고 multi_class 매개변수의 기본값은 auto이다. auto로 지정됨은 solver="liblinear"일 경우를 제외하고 default value가 multinomial이 된다는 의미다(solver="liblinear일 경우에는 multi_class="ovr"가 된다).

In [14]:
auto = LogisticRegression(random_state=1)

auto.fit(features_standardized, target)
new_observation = [[.5, .5, .5, .5]]
print(auto.predict(new_observation), auto.predict_proba(new_observation))

[1] [[0.01982185 0.74491886 0.23525928]]


결과를 보면 분류는 class 1으로 했으며 각 class에 속할 확률을 0부터 순서대로 나타내고 있다.

### 규제로 분산 줄이기 
로지스틱 회귀 모델의 분산을 줄여야 한다. -> 규제 강도를 조절하는 하이퍼파라미터 C를 사용한다.

In [6]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

#데이터
iris = datasets.load_iris()
features = iris.data
target = iris.target

#표준화
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

#로지스틱 회귀 모델
logistic_regression_l1 = LogisticRegressionCV(Cs=10, random_state=2020, n_jobs=-1)
logistic_regression_l2 = LogisticRegressionCV(penalty='l2', Cs=10, random_state=2020, n_jobs=-1)


model1 = logistic_regression_l1.fit(features_standardized, target)
model2 = logistic_regression_l2.fit(features_standardized, target)
new_observation = [[.5, .5, .5, .5]]
print(model1.predict(new_observation), model1.predict_proba(new_observation))
print(model2.predict(new_observation), model2.predict_proba(new_observation))

[1] [[5.96244929e-04 9.70140320e-01 2.92634349e-02]]
[1] [[5.96244929e-04 9.70140320e-01 2.92634349e-02]]


규제(penalty)는 alpha 값으로 페널티를 부여해 회귀 계수 값의 크기를 감소시켜 과적합을 개선하는 방식이다. 규제는 l1과 l2로 두 가지가 있는데 l1은 라쏘를 의미하고 l2는 릿지를 의미한다.  
라쏘(l1)의 경우, alapha * $|W|_1$을 의미하며 비용 함수 RSS(W) + alapha * $|W|_1$식을 최소화하는 W를 찾는 게 목표다(W의 절댓값에 페널티).  
$RSS(w_0, w_1, ..., w_n) = \frac{1}{N}\sum(y_i-\hat{y})^2$.  
릿지(l2)의 경우, alapha * $|W|_2^2$을 의미하며 비용 함수 RSS(W) + alapha * $|W|_2^2$식을 최소화하는 W를 찾는 게 목표다(W의 제곱에 페널티). 
- alpha 값을 크게 하면 비용함수는 회귀 계수의 W의 값을 작게 해서 과적합을 개선할 수 있다. 반대로 회귀 계수 W의 값이 커져도 alpha값을 작게 해서 상쇄를 해서 학습 데이터 적합을 개선할 수 있다.
- 라쏘 회귀의 경우는 회귀 계수를 0으로 만들지만 릿지 회귀의 경우 회귀 계수를 0으로 만들지는 않는다. 0으로 만든다는 의미는 불필요한 회귀 계수를 0으로 감소시켜 제거한다. 


__Q. |W|의 식이 어떻게 되는지 제대로 설명하자.__

### 대용량 데이터에서 분류기 훈련하기  
아주 많은 데이터셋에서 간단한 분류 모델을 훈련한다. -> 사이킷런의 LogisticRegression에서 solver 매개변수를 확률적 평균 경사 하강법(saga)으로 지정.  

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data
target = iris.target

scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

logistic_regression = LogisticRegression(random_state=0, solver="saga")

model = logistic_regression.fit(features_standardized, target)

- 사이킷런의 LogisticRegression 클래스는 solver 매개변수에서 여러 가지 로지스틱 회귀 훈련 기법 지원.
- 확률적 평균 경사 하강법(saga)은 가장 빠르게 모델을 훈련할 수 있는 방법.
- __solver 매개변수에 'saga'를 지정하고 penalty를 'elasticnet'으로 지정하고 l1_ratio를 조정하면 l1에는 그대로 페널티가 적용되고 1에서 뺀 나머지를 l2에 적용이 된다__(penalty에 'none'으로 하면 페널티를 부여하지 않을 수 있다).

### 불균형한 클래스 다루기
간단한 분류 모델을 훈련해야 한다. -> 사이킷런의 logisticRegression 클래스로 로지스틱 회귀 모델 훈련시키기.

In [5]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
features = iris.data
target = iris.target

# 클래스0의 개수를 10개로 줄이고 클래스1, 2를 합쳐 클래스 1의 개수를 100개로 만들었다.
features = features[40:,:]
target = target[40:]

target = np.where((target == 0), 0, 1)   
"""np.where은 for문 보다 빠르며 조건문을 사용한다. 근본적으로 인덱스를 반환하지만 조건에 해당하는 값은 첫번째 값으로 변환하고 
해당하지 않는 값은 두 번째 값으로 변환한다. 위에서 1대신에 target으로 썼으면 값이 그대로였을 것이다. """

sacler = StandardScaler()
features_standardized = scaler.fit_transform(features)

logistic_regression = LogisticRegression(random_state=20200924, class_weight="balanced")
#logistic_regression = LogisticRegression(random_state=20200924, class_weight={0:10, 1:1})   #class_weight 매개변수에 {클래스 레이블: 가중치} 형식의 딕셔너리를 전달 가능.

model = logistic_regression.fit(features_standardized, target)

LogisticRegression은 자체적으로 불균형한 클래스를 다루 수 있다. 전처리 과정을 통해 불균형한 클래스가 균형이 잡히지 않았다면 class_weight매개변수에 "balanced"를 지정하면 클래스 빈도의 역수로 가중치를 부여한다.  
$w_j = \frac{n}{kn_J}$,  $w_j$: 클래스j의 가중치, n:전체 샘플 개수, n_j: 클래스 j에 있는 샘플 개수, K: 전체 클래스 개수.

In [7]:
from sklearn.utils.class_weight import compute_class_weight

# 클래스 레이블이 0, 1인 데이터의 클래스 가중치 계산하기
compute_class_weight('balanced', [0,1], target)

array([5.5 , 0.55])

결과를 통해서 앞으로 클래스0은 가중치 5.5가 지정되고, 클래스 1에는 가중치 0.55가 지정됨을 알 수 있다.